# 1. Frame the problem
Using the customer description, Define the problem your trying to solve in your own words (remember this is not technial but must be specific so the customer understands the project

Using machine learning and deep learning, identify handwritten cursive letters.

# 2. Get the Data 
Define how you recieved the data (provided, gathered..)

In [1]:
!pip3 install opencv-python

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
import os

In [3]:
# import zipfile

# with zipfile.ZipFile("Cursive.zip", 'r') as zip_ref:
#     zip_ref.extractall()

# 3. Explore the Data
Gain insights into the data you have from step 2, making sure to identify any bias

Some things I'm thinking about:
- black and white
- scale images
- canny edges

In [4]:
from pathlib import Path
import cv2
import glob

In [5]:
all_files = glob.glob('Cursive/**', recursive=True)

file_extensions = set()
for file_path in all_files:
   if os.path.isfile(file_path):
       _, extension = os.path.splitext(file_path)
       if extension:
           file_extensions.add(extension)
print("Unique file extensions:", file_extensions)

Unique file extensions: {'.heic', '.HEIC', '.JPG', '.jpg', '.jpeg', '.png', '.PNG'}


In [6]:
# look for types of file endings
png_count = len(glob.glob('Cursive/**/*.png', recursive=True))

for i in file_extensions:
    ext_counter = len(glob.glob(f'Cursive/**/*{i}', recursive=True))
    print(f"{i}: {ext_counter}")

.heic: 41
.HEIC: 415
.JPG: 104
.jpg: 106
.jpeg: 36
.png: 78
.PNG: 52


In [7]:
import os

# file folder counts
dirs = 0
files = 0

for root, dirnames, filenames in os.walk("Cursive"):
    dirs += len(dirnames)
    files += len(filenames)

print(dirs)
print(files)

39
900


In [8]:
from collections import Counter

counts = Counter()

for f in Path("Cursive").rglob("*"):
        if f.is_file():
            counts[str(f.parent)] += 1

bad_folders = []
for folder, num_files in sorted(counts.items()):
        if num_files != 26:
            print(f"{folder:} | {num_files}")
            bad_folders.append(folder)

Cursive | 1
Cursive/S10/S13 | 76
Cursive/S11 | 2
Cursive/S11/.ipynb_checkpoints | 2
Cursive/S14 | 5
Cursive/S17 | 25
Cursive/S33 | 25
Cursive/S34 | 10


## Conclusions

I need to:
- Label Data
- Crop images (content aware)
- Rotate (not sure)
- Black and white (opencv)
- Canny edge (highlight edges so the letters are easier to pick out)

# 4.Prepare the Data


Apply any data transformations and explain what and why


In [9]:
# label every handwriting thing as a-z and append index (a_4)
# exclude folders with counts != 26

all_folders = [f for f in glob.glob('Cursive/**', recursive=True) if os.path.isdir(f)]
good_folders = [x for x in all_folders if x not in list(bad_folders) and x != "Cursive/"]
print(good_folders, len(all_folders), len(good_folders))

['Cursive/S19', 'Cursive/S35', 'Cursive/S29', 'Cursive/S10', 'Cursive/S10/S11', 'Cursive/S10/S14', 'Cursive/S10/S12', 'Cursive/S26', 'Cursive/S16', 'Cursive/S18', 'Cursive/S20', 'Cursive/S12', 'Cursive/S21', 'Cursive/S2', 'Cursive/S3', 'Cursive/S28', 'Cursive/S32', 'Cursive/S27', 'Cursive/S1', 'Cursive/s15', 'Cursive/S23', 'Cursive/S4', 'Cursive/S30', 'Cursive/S9', 'Cursive/S13', 'Cursive/S5', 'Cursive/S7', 'Cursive/S25', 'Cursive/S31', 'Cursive/S24', 'Cursive/S8', 'Cursive/S6'] 39 32


In [38]:
import os
import glob
import string
import shutil


letters = string.ascii_lowercase + string.ascii_uppercase
 
src_root = ""
dst_root = "LabeledData"

good_folders = sorted(good_folders, key=lambda x: (len(x), x))

os.makedirs(dst_root, exist_ok=True)
folder_to_label = {folder: letters[i % len(letters)] for i, folder in enumerate(good_folders)}

for idx, folder_name in enumerate(good_folders):
    src_folder = os.path.join(src_root, folder_name)
    dst_folder = os.path.join(dst_root, folder_name)
    os.makedirs(dst_folder, exist_ok=True)

    files = sorted(
        [f for f in glob.glob(os.path.join(src_folder, "*"), recursive=True) if os.path.isfile(f)],
        key=lambda x: os.path.basename(x)
    )

    if not files:
        print("no file")
        continue

    folder_label = folder_to_label[folder_name]

    for j, path in enumerate(files):
        label = letters[j % len(letters)]
        ext = os.path.splitext(path)[1]
        new_name = f"{label}_{idx}{ext}"
        dst_path = os.path.join(dst_folder, new_name)
        shutil.copy2(path, dst_path)

    print(f"copied {len(files)} from {folder_name}")


copied 26 from Cursive/S1
copied 26 from Cursive/S2
copied 26 from Cursive/S3
copied 26 from Cursive/S4
copied 26 from Cursive/S5
copied 26 from Cursive/S6
copied 26 from Cursive/S7
copied 26 from Cursive/S8
copied 26 from Cursive/S9
copied 26 from Cursive/S10
no file
no file
copied 26 from Cursive/S16
copied 26 from Cursive/S18
copied 26 from Cursive/S19
copied 26 from Cursive/S20
copied 26 from Cursive/S21
copied 26 from Cursive/S23
copied 26 from Cursive/S24
copied 26 from Cursive/S25
copied 26 from Cursive/S26
copied 26 from Cursive/S27
copied 26 from Cursive/S28
copied 26 from Cursive/S29
copied 26 from Cursive/S30
copied 26 from Cursive/S31
copied 26 from Cursive/S32
no file
copied 26 from Cursive/s15
copied 26 from Cursive/S10/S11
copied 26 from Cursive/S10/S12
copied 26 from Cursive/S10/S14


In [31]:
!pip3 install pillow_heif

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 11.3 MB/s eta 0:00:00a 0:00:01


In [40]:
import shutil
from pathlib import Path

root = Path("LabeledData/Cursive")

count = 0
for checkpoint_dir in root.rglob(".ipynb_checkpoints"):
    if checkpoint_dir.is_dir():
        shutil.rmtree(checkpoint_dir)
        count += 1
print(count)

0


In [43]:
from PIL import Image
from pillow_heif import register_heif_opener

register_heif_opener()

os.makedirs("Exploded", exist_ok=True)

formatting = 'png'

all_files = list(Path("LabeledData/Cursive").rglob("*.*"))

for i, f in enumerate(all_files):
    with Image.open(f) as img:
        out_path = f"Exploded/{f.stem}.png"
        img.save(out_path, "PNG")
    if i % 26 == 0:
        print(f"saved {i+1}/{len(all_files)} — {f.name}")

ERROR! Session/line number was not unique in database. History logging moved to new session 13
saved 1/702 — n_14.jpg
saved 27/702 — j_23.PNG
saved 53/702 — k_9.JPG
saved 79/702 — n_20.HEIC
saved 105/702 — x_12.jpeg
saved 131/702 — q_13.HEIC
saved 157/702 — h_15.png
saved 183/702 — t_16.jpg
saved 209/702 — f_1.HEIC
saved 235/702 — t_2.HEIC
saved 261/702 — a_22.PNG
saved 287/702 — f_26.png
saved 313/702 — h_21.JPG
saved 339/702 — u_0.HEIC
saved 365/702 — f_28.HEIC
saved 391/702 — g_17.HEIC
saved 417/702 — q_3.jpg
saved 443/702 — w_24.JPG
saved 469/702 — y_8.HEIC
saved 495/702 — d_4.heic
saved 521/702 — x_6.JPG
saved 547/702 — u_19.png
saved 573/702 — h_25.HEIC
saved 599/702 — u_18.HEIC
saved 625/702 — r_5.HEIC
saved 651/702 — e_29.jpg
saved 677/702 — o_31.HEIC


In [ ]:
discrepencies = []

In [ ]:
all_files = list(Path("Exploded").rglob("*.*"))

os.makedirs("Preprocess", exist_ok=True)

for f in all_the_files:
    p = cv2.imread(str(f))
    gray = cv2.cvtColor(p, cv2.COLOR_BGR2GRAY)
    canny = cv2.Canny(gray, 50, 150)
    cv2.imwrite(f'Preprocess/{f.name}', canny)

In [12]:
!pip install pyCAIR
!pip install natsort

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [13]:
from pyCAIR.seam_carve import cropByColumn

seam_img, col_cropped = cropByColumn(image=image, display_seams=1, lsit=lsit)

NameError: name 'image' is not defined

In [ ]:
# scale images
# https://www.geeksforgeeks.org/python/resize-multiple-images-using-opencv-python/


# 5. Model the data
Using selected ML models, experment with your choices and describe your findings. Finish by selecting a Model to continue with

## Conclusions

# 6. Fine Tune the Model

With the select model descibe the steps taken to acheve the best rusults possiable 


# 7. Present
In a customer faceing Document provide summery of finding and detail approach taken


# 8. Launch the Model System
Define your production run code, This should be self susficent and require only your model pramaters 


In [ ]:
# import libraries
import pandas as pd
import numpy as np
import joblib
from scipy.sparse import hstack, csr_matrix


def infrence(text):

    # load everything in
    model = joblib.load("./models/best_mnb.pkl")
    vectorizer = joblib.load("./models/vectorizer.pkl")
    scaler = joblib.load("./models/scaler.pkl")
    feature_params = joblib.load("./models/feature_params.pkl")
    
    s_words = feature_params[0]
    h_words = feature_params[1]
    good_emails = feature_params[2]

    # clean and tokenize the text
    cleaned = re.sub(r'[^A-Za-z\s]', '', text).lower()
    tokens = [word for word in cleaned.split() if word not in stopwords and len(word) > 2]
    token_counter = collections.Counter(tokens)

    # freq of most common words
    spam_freq = np.sum([token_counter[word.lower()] for word in s_words])
    ham_freq = np.sum([token_counter[word.lower()] for word in h_words])

    # amt of punctuation
    punctuation_chars = string.punctuation
    regex_pat = "[" + re.escape(punctuation_chars) + "]"
    punct_freq = len(re.findall(regex_pat, text))

    # good email domain
    good_email_count = sum(1 for x in good_emails if x in text.lower())

    # msg length
    msg_len = len(text)

    # create df of numeric
    numeric_features = pd.DataFrame([[spam_freq, punct_freq, msg_len, ham_freq, good_email_count]],
                                columns=['spam_most_common_freq','punctuation_freq','msg_len','ham_most_common_freq','good_email'])
    numeric_features_scaled = scaler.transform(numeric_features)

    # vectorize words
    text_features = vectorizer.transform([text])
    X = hstack([text_features, csr_matrix(numeric_features_scaled)])
    
    # predcit!
    prediction = model.predict(X)
        
    return prediction[0]

In [ ]:
test1 = '''Subject: stock market information

sender : trinity venture , inc . address : 1911 westmead # 2710 , houston , tx 7707 call fax toll free : 877-393 - 7237 hours : 9 5 pm cst hi , receive e-mail address someone interest stock market information . e-mail informational purpose . offer sell solicit security . wish receive type information , please click " reply " put " remove " subject . permanently remove address our database . news help due diligence abrg , www . yahoo . com click " stock quote " . put abrg symbol lookup area click " quote " . happy invest ! - dean casia president trinity venture , inc . news release ambra resources group inc . # 610-800 west pender street vancouver , b . c . canada v6c 2v6 symbol abrg ( otc : bb ) acquisition venture oil & gas , inc . ambra resources group inc . adds projects ambra vancouver , british columbium - 4 , 1999 ambra resource group inc . ( otc bb : abrg ) , acquire 50 % ownership venture oil & gas , inc . ambra become 50 % owner property project venture interest . consideration ambra 's purchase 50 % capital stock venture inc . one million shares ambra common stock . part acquisition , venture assign ambra , 50 % interest bastian bay field prospect , state lease " 9800 " . 1 , plaquemine parish , louisiana . additional project assign ambra venture ambra shall right request assignment 50 % interest retain venture various project . primary business venture inc . acquire oil gas property re-mediation re-completion work result enhance recovery rate bring back commercial production . current economic condition petroleum industry facilitate acquisition property larger produce company declare surplus property . ambra venture inc . favorable position able acquire property while inventory offering high level . venture currently negotiate acquisition multiple oil gas project texa , louisiana oklahoma , party anticipate add significantly ambra 's resource base productive project . board director john m . hickey , president contact : ambra resource group inc . investor relation : 800-698 - 3377 604-669 - 2723 web site : http : / / www . ambraresource . com release informational purpose . offer sell solicit security product kind . release include information constitute forward-look statement pursuant safe harbor provision private security litigation reform act 1995 . forward-look statement involve risk uncertainty cause actual result differ materially future result encompass within forward-look statement . material provide ambra resource group inc . " " basis . ambra resource group inc . expressly disclaim warranty , express imply , include without limitation , warranty merchantability fitness particular purpose , respect service material product . event shall ambra resource group inc . liable direct , indirect , incidental , punitive consequential damages kind whatsoever respect service , material product . trinity venture , inc . receive fee $ 5 , 0 distribute document . permanently remove e-mail address our file call fax us toll free 877-393 - 7237 ; click " reply " put " remove " subject .
'''
pred = infrence(test1)
print("spam" if pred == 1 else "ham")

In [ ]:
test2 = '''Subject: postdoc position groningen , netherland

job position : postdoc dept . social pharmacy pharmacoepidemiology , groningen institute drug study , faculty mathematic natural science , netherland . description work group social pharmacy pharmacoepidemiology perform fundamental apply research epidemiological medical literature database order determine effectiveness / side-effect profile drug . post-doc ask participate program develop computer text analysis pattern recognition technique extraction ( side ) effect profile drug pharmaceutical medical electronic literature database : ( 1 ) source information lead innovative drug research ; ( 2 ) determine benefit-risk profile drug . phd - student assign program . requirement computer linguist computer scientist , complete phd - project expertise corpus linguistics , mathematical linguistics intelligent information retrieval ; interest pharmaceutical science innovative drug research ; expertise datum mine pattern recognition method desire . remark salary basis ministry guideline minimum dfl . 3844 , - maximum dfl . f . 7 . 125 , - ( schaal 10 / 11 rwoo ) bruto pro month , dependent education experience . work group social pharmacy pharmacoepidemiology part dutch school ' groningen - utrecht institute drug exploration ' ( guide ) , acknowledge royal dutch academy science . appointment two . information project : prof . dr . r . vo , email : r . vo @ farm . rug . nl ; tel . + 31 . 50 . 3633331 / 3633272 ; fax . + 31 . 50 . 3633311 . reaction . s . . p . , preferably before july 1st , 1997 . - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - marc weeber http : / / www . farm . rug . nl / marc / home . html groningen university centre pharmacy marc @ farm . rug . nl social pharmacy pharmacoepidemiology tel : + 31 50 3637571 _ _ _ . deusinglaan 2 fax : + 31 50 3633311 | 9713 aw groningen , netherland - - - - - - - - - - - - - - - - - - - - - 0 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
'''
pred = infrence(test2)
print("spam" if pred == 1 else "ham")